# Library

In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import time

# Load Model

In [2]:
# Path to the folder containing the models
model_folder = "Modelling MobileNet/Model MobileNet"

# List all files in the folder
model_files = os.listdir(model_folder)

# Filter only files with .h5 extension (TensorFlow model files)
model_files = [file for file in model_files if file.endswith('.h5')]

# Load all models into a list
models = []
for model_file in model_files:
    model_path = os.path.join(model_folder, model_file)
    # Remove the file extension to use as the model name
    model_name = os.path.splitext(model_file)[0]
    model = load_model(model_path, compile=False)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    models.append((model_name, model))  # Store both model name and model object in the list

In [3]:
# Check apakah model yang di-load sudah benar
# Get name all models
models

[('MobileNet_Basic', <keras.engine.functional.Functional at 0x1fac416ec50>),
 ('MobileNet_Modified075',
  <keras.engine.functional.Functional at 0x1fad7b1c910>),
 ('MobileNet_Modified076',
  <keras.engine.functional.Functional at 0x1faf82815a0>),
 ('MobileNet_Modified077',
  <keras.engine.functional.Functional at 0x1fb02024970>),
 ('MobileNet_Modified078',
  <keras.engine.functional.Functional at 0x1fb06a229e0>),
 ('MobileNet_Modified079',
  <keras.engine.functional.Functional at 0x1fb0698c850>),
 ('MobileNet_Modified080',
  <keras.engine.functional.Functional at 0x1fb06c43a00>),
 ('MobileNet_Modified081',
  <keras.engine.functional.Functional at 0x1fb06cd45e0>),
 ('MobileNet_Modified082',
  <keras.engine.functional.Functional at 0x1fb06d686a0>),
 ('MobileNet_Modified083',
  <keras.engine.functional.Functional at 0x1fbec48e050>),
 ('MobileNet_Modified084',
  <keras.engine.functional.Functional at 0x1fb0200a500>),
 ('MobileNet_Modified085',
  <keras.engine.functional.Functional at 0x1fb

# Load Dataset

In [4]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
        "Dataset/Test",  # Path direktori dataset validasi
        target_size=(224, 224),  # Resolusi gambar
        classes=['Negative', 'Positive'],  # Kelas target
        class_mode='categorical',  # Mode kelas (categorical karena ada lebih dari 2 kelas)
        shuffle=False,
        batch_size=1) # Tidak perlu diacak karena kita ingin mencocokkan hasil dengan label
        

Found 8000 images belonging to 2 classes.


# Count Runtime 

In [5]:
# Ukur runtime inferensi untuk setiap model dan satu dataset keseluruhan terlebih dahulu baru ganti model
inference_times = [[] for _ in range(len(models))]

# Iterate over models
for j, (model_name, model) in enumerate(models):
    total_batches = len(val_generator)
    total_images = total_batches * val_generator.batch_size
    total_predictions = 0
    
    # Iterate over images in the validation dataset
    for i, (images, _) in enumerate(val_generator):
        # Measure inference time for the entire dataset and append to the corresponding model's list
        start_time = time.time()
        predictions = model.predict(images, verbose=0)
        end_time = time.time()
        inference_time = end_time - start_time
        inference_times[j].append(inference_time)
        
        total_predictions += len(predictions)
        
        # Display message when done predicting entire dataset
        if total_predictions >= total_images:
            print(f"Model {model_name}: Done Inference for Dataset ({total_images} images)")
            break

Model MobileNet_Basic: Done Inference for Dataset (8000 images)
Model MobileNet_Modified075: Done Inference for Dataset (8000 images)
Model MobileNet_Modified076: Done Inference for Dataset (8000 images)
Model MobileNet_Modified077: Done Inference for Dataset (8000 images)
Model MobileNet_Modified078: Done Inference for Dataset (8000 images)
Model MobileNet_Modified079: Done Inference for Dataset (8000 images)
Model MobileNet_Modified080: Done Inference for Dataset (8000 images)
Model MobileNet_Modified081: Done Inference for Dataset (8000 images)
Model MobileNet_Modified082: Done Inference for Dataset (8000 images)
Model MobileNet_Modified083: Done Inference for Dataset (8000 images)
Model MobileNet_Modified084: Done Inference for Dataset (8000 images)
Model MobileNet_Modified085: Done Inference for Dataset (8000 images)
Model MobileNet_Modified086: Done Inference for Dataset (8000 images)
Model MobileNet_Modified087: Done Inference for Dataset (8000 images)
Model MobileNet_Modified08

In [6]:
# Hitung rata-rata runtime inferensi untuk setiap model
average_inference_times = [np.mean(times) for times in inference_times]

# Print hasil
for i, model_file in enumerate(model_files):
    print(f"Model {model_file}: Rata-rata runtime inferensi per gambar: {average_inference_times[i]} seconds")

Model MobileNet_Basic.h5: Rata-rata runtime inferensi per gambar: 0.051348755657672884 seconds
Model MobileNet_Modified075.h5: Rata-rata runtime inferensi per gambar: 0.046057122975587846 seconds
Model MobileNet_Modified076.h5: Rata-rata runtime inferensi per gambar: 0.0450742963552475 seconds
Model MobileNet_Modified077.h5: Rata-rata runtime inferensi per gambar: 0.04550249469280243 seconds
Model MobileNet_Modified078.h5: Rata-rata runtime inferensi per gambar: 0.04579774731397629 seconds
Model MobileNet_Modified079.h5: Rata-rata runtime inferensi per gambar: 0.04542930924892426 seconds
Model MobileNet_Modified080.h5: Rata-rata runtime inferensi per gambar: 0.04587820717692375 seconds
Model MobileNet_Modified081.h5: Rata-rata runtime inferensi per gambar: 0.04620877134799957 seconds
Model MobileNet_Modified082.h5: Rata-rata runtime inferensi per gambar: 0.04564696231484413 seconds
Model MobileNet_Modified083.h5: Rata-rata runtime inferensi per gambar: 0.04597196352481842 seconds
Model

# Wrap Up

In [8]:
# Make dataset for model name and average inference time
# Initialize an empty list to store the model name and average inference time tuples
model_inference_times = []

# Iterate over models and average inference times
for (model_name, _), avg_time in zip(models, average_inference_times):
    # Append the model name and average inference time tuple to the list
    model_inference_times.append((model_name, avg_time))

# Create a DataFrame from the list of tuples
df = pd.DataFrame(model_inference_times, columns=['Model Name', 'Average Inference Time'])

# Print the DataFrame
print(df)

               Model Name  Average Inference Time
0         MobileNet_Basic                0.051349
1   MobileNet_Modified075                0.046057
2   MobileNet_Modified076                0.045074
3   MobileNet_Modified077                0.045502
4   MobileNet_Modified078                0.045798
5   MobileNet_Modified079                0.045429
6   MobileNet_Modified080                0.045878
7   MobileNet_Modified081                0.046209
8   MobileNet_Modified082                0.045647
9   MobileNet_Modified083                0.045972
10  MobileNet_Modified084                0.045910
11  MobileNet_Modified085                0.046241
12  MobileNet_Modified086                0.047194
13  MobileNet_Modified087                0.046847
14  MobileNet_Modified088                0.047529
15  MobileNet_Modified089                0.047133
16  MobileNet_Modified090                0.048257
17  MobileNet_Modified091                0.047472
18  MobileNet_Modified092                0.048159


In [9]:
# Export the DataFrame to a CSV file
df.to_csv('inference times MobileNet.csv', index=False)